<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/07_document_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 単語埋め込みを使った文書分類
* 今回も、IMDbデータセットの感情分析を文書分類問題として解く。
* ただし今回は、fastTextのような学習済みの単語埋め込みは使わない。
* 単語埋め込み自体の学習も、ネットワークの学習と同時におこなう。
* IMDbデータの準備も、`torch.torchtext`を使っておこなう。
 * つまりすべてをPyTorchのなかでおこなう。
* 参考資料
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

## データをどう扱うか
* ネットワークへの入力は、単語埋め込みを、単語の出現順どおりに並べた列にする。
 * ミニバッチは[ミニバッチのなかでの最大文書長, ミニバッチのサイズ, 単語埋め込み次元数]という形の3階のテンソルになる。
* そして、前向き計算のなかではじめて、単語埋め込みの平均をとることにする。
 * `.mean(0)`と、軸0で平均をとることになる。


## 07-00 Google Colabのランタイムのタイプを変更する
* Google ColabのランタイムのタイプをGPUに変更しておこう。
 * 上のメニューの「ランタイム」→「ランタイムのタイプを変更」→「ハードウェア　アクセラレータ」から「GPU」を選択

## 07-01 torchtextを使ってIMDbデータを読み込む
* ここでIMDbデータセットの読み込みにつかう`torchtext.datasets`については、下記を参照。
 * https://torchtext.readthedocs.io/en/latest/datasets.html

### 実験の再現性確保のための設定など
* https://pytorch.org/docs/stable/notes/randomness.html

In [ ]:
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
from torchtext.data import Field, LabelField, BucketIterator

SEED = 123

random.seed(SEED)
torch.manual_seed(SEED)
torch.set_deterministic(True)

### torchtextのフィールド
* TEXTフィールドと、LABELフィールドという２種類のFieldオブジェクトのインスタンスを作る。
 * Fieldクラスの詳細については[ここ](https://github.com/pytorch/text/blob/master/torchtext/data/field.py)を参照。
* TEXTフィールドは、テキストの前処理の仕方を決めておくのに使う。
 * tokenizerは、デフォルトでは単にstring型のsplitメソッドを適用するだけになる。これは高速だが、tokenizationとしては雑。
* LABELフィールドは、ラベルの前処理に使う。

In [ ]:
TEXT = Field(tokenize="spacy")
LABEL = LabelField()

### IMDbデータセットをダウンロードした後、前処理しつつ読み込む
* ダウンロードはすぐ終わるが、解凍に少し時間がかかる。
* また、TEXTフィールドでspaCyのtokenizationを使うように設定したので、少し時間がかかる。
 * string型のsplitメソッドでtokenizeすると、時間はあまりかからない。（そのかわり、やや雑なtokenizationになる。）

In [ ]:
train_valid_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

### 最初の文書を見てみる

In [ ]:
print(train_valid_data[0].text)

In [ ]:
print(train_valid_data[0].label)

### テストセット以外の部分を訓練データと検証データに分ける

In [ ]:
train_data, valid_data = train_valid_data.split(split_ratio=0.8)

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

In [ ]:
print(train_data[0].text)

### データセットの語彙とラベルを作る
* TEXTラベルのほうでは、最大語彙サイズを指定する。

In [ ]:
MAX_VOCAB_SIZE = 25000 # この値は適当。

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

なぜ語彙サイズが25,000ではなく25,002なのかについては、少し下の説明を参照。

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

### 出現頻度順で上位２０単語を見てみる

In [ ]:
print(TEXT.vocab.freqs.most_common(20))

### 単語ID順に最初の１０単語を見てみる
* IDのうち、0と1は、未知語とパディング用の単語という特殊な単語に割り振られている。
 * 未知語は`<unk>`という特殊な単語に置き換えられる。これのIDが0。
 * パディングとは、長さが不揃いの複数の文書を同じミニバッチにまとめるとき、すべての文書の長さを無理やりそろえるため、文書末尾に特殊な単語（元々の語彙にない、人工的に用意した単語）を追加すること。
 * パディング用の単語が`<pad>`になっているのは、上のほうで使ったFieldクラスのインスタンスを作るときのデフォルトの値がこの`<pad>`になっているため。

In [ ]:
print(TEXT.vocab.itos[:10])

### ラベルのほうのIDを確認する
* こちらはnegとposに対応する２つのIDしかない。

In [ ]:
print(LABEL.vocab.stoi)

### ミニバッチを取り出すためのiteratorを作る
* ミニバッチのサイズを指定する。
 * ミニバッチのサイズは、性能を出すためにチューニングすべきハイパーパラメータのひとつ。

In [ ]:
BATCH_SIZE = 100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator = data.BucketIterator(train_data, batch_size=BATCH_SIZE, device=device,
                                     sort_within_batch=True, shuffle=True, sort_key=lambda x: len(x.text))
valid_iterator = data.BucketIterator(valid_data, batch_size=BATCH_SIZE, device=device)
test_iterator = data.BucketIterator(test_data, batch_size=BATCH_SIZE, device=device)

### ミニバッチの中身を確認する

* 訓練データのiteratorを回してミニバッチをすべて取得してみる
 * ミニバッチのshapeを表示してみる

In [ ]:
for i, batch in enumerate(train_iterator):
  print(i, batch.text.shape)

* ミニバッチの形は、[ミニバッチ内での最大文書長, ミニバッチのサイズ]になっていることに注意！
 * ミニバッチのサイズが最初に来ているのではない！
 * [ミニバッチのサイズ, ミニバッチ内での最大文書長]という形にしたいなら、テキストのfieldを作るとき以下のようにする。

__`TEXT = data.Field(tokenize="spacy", batch_first=True)`__

* 上記のループを抜けたあとには、変数batchには最後のミニバッチが代入されている。

In [ ]:
batch.text.shape

* このミニバッチに含まれる文書のうち、最初の文書の単語ID列を表示させてみる。

In [ ]:
print(batch.text[:, 0])

* このミニバッチに含まれる文書のうち、最初の文書の単語ID列を単語列に戻したものを表示させてみる。

In [ ]:
print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:, 0]]))

* このミニバッチに含まれる文書のうち、最後の文書の単語ID列を表示させてみる。

In [ ]:
print(batch.text[:, BATCH_SIZE-1])

最後の文書の末尾は「1」で埋められていることが分かる。

この1は、パディング用単語のIDだったことを想起されたい。

ミニバッチに含まれる文書の長さを調べると、文書が文書長の降順に並べられていることが分かる。

In [ ]:
(batch.text != 1).sum(0)

## 07-02 MLPによる文書分類の準備
* 今回は、ごく簡単なMLPで文書分類をする。
* 文書中の全単語トークンの埋め込みベクトルの平均を、MLPの入力とする。
 * 当然、語順の情報は使われない。
 * つまり、bag-of-wordsモデルになっている。

### 定数の設定
* 単語埋め込みベクトルの次元数は128にする。

In [ ]:
INPUT_DIM = len(TEXT.vocab)
NUM_CLASS = len(LABEL.vocab)
EMBED_DIM = 128
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

print(f'語彙サイズ {INPUT_DIM}, クラス数 {NUM_CLASS}, 単語埋め込み次元 {EMBED_DIM}')

### モデルを定義する前にPyTorchの単語埋め込みがどんなものかを見てみる
* 埋め込みとは、単語IDから単語ベクトルへのマッピング。

* 以下のように、語彙サイズと埋め込みの次元数を指定しつつ、torch.nn.Embeddingのインスタンスを作ればよい。

In [ ]:
embed = nn.Embedding(INPUT_DIM, EMBED_DIM, padding_idx=PAD_IDX)

* パディング用の単語の埋め込みはゼロベクトルになる。

In [ ]:
print(embed(torch.tensor([21,1])))

* 埋め込みの効果を見てみる

In [ ]:
for i, batch in enumerate(train_iterator):
  print(i, embed(batch.text).shape)

### モデルの定義
* MLP（多層パーセプトロン）だが、入り口に単語埋め込み層が挿入されている。

In [ ]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx):
    super(EmbedTextSentiment, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)

  def forward(self, text):
    x = self.embed(text)
    x = x.mean(0) # 文書に含まれる全単語トークンの単語ベクトルの平均
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

### モデルを作る
* モデル（のインスタンス）をGPUに移動させている点に注意。

In [ ]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX).to(device)

### 損失関数とoptimizerとschedulerを作る

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

### 訓練用の関数
* 最初の`model.train()`に注意。こうやって、モデルを訓練モードに設定する。
 * 例えば、dropoutを含むモデルなど、訓練時と評価時で、ふるまい方を変える必要があるときがあるため、こういうことをする。

In [ ]:
def train(data_iterator, model, optimizer, scheduler, criterion):

  model.train()

  train_loss = 0
  train_acc = 0
  for batch in data_iterator:
    optimizer.zero_grad()
    text, cls = batch.text, batch.label
    output = model(text)
    loss = criterion(output, cls)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    train_acc += (output.argmax(1) == cls).float().mean().item()

  scheduler.step()

  num_batch = len(data_iterator)
  return train_loss / num_batch, train_acc / num_batch

### 評価用の関数
* 最初の`model.eval()`に注意。こうやって、モデルを評価モードに設定する。
 * 例えば、dropoutを含むモデルなど、訓練時と評価時で、ふるまい方を変える必要があるときがあるため、こういうことをする。

In [ ]:
def test(data_iterator, model, criterion):

  model.eval()

  loss = 0
  acc = 0
  for batch in data_iterator:
    text, cls = batch.text, batch.label
    with torch.no_grad():
      output = model(text)
      loss = criterion(output, cls)
      loss += loss.item()
      acc += (output.argmax(1) == cls).float().mean().item()

  num_batch = len(data_iterator)
  return loss / num_batch, acc / num_batch

## 07-03 分類器の訓練と検証セットでの評価

In [ ]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

## 07-04 再検討
* 訓練データ上での分類精度がほぼ100%になってしまっている。
* 検証データでの分類精度と大きな差があり、明らかにオーバーフィッティング。

### ドロップアウトを使う
* モデルのインスタンスを作るときにdropoutの確率を引数pで指定できるようにする。

In [ ]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx, p=0.0):
    super(EmbedTextSentiment, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.dropout = nn.Dropout(p=p)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)

  def forward(self, text):
    x = self.dropout(self.embed(text)) #埋め込み層の直後にdropout
    x = x.mean(0)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [ ]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

### L２正則化を使う
* optimizerのweight_decayパラメータを0より大きな値にする。

In [ ]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [ ]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

### early stopping
* dev setでのaccuracyが4回連続で最高値を下回ったら訓練を終えることにする。
* early stoppingの実現については、PyTorch Lightningを使う手もある。
 * https://pytorch-lightning.readthedocs.io/en/latest/early_stopping.html

In [ ]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [ ]:
patience = 4
early_stop_count = 0
best_valid_acc = 0.0

MIN_N_EPOCHS = 10 # 最低このエポック数は実行する
N_EPOCHS = 50 # エポック数を増やしておく
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

  # early stopping
  if epoch + 1 > MIN_N_EPOCHS:
    if best_valid_acc <= valid_acc:
      best_valid_acc = valid_acc
      early_stop_count = 0
    else:
      early_stop_count += 1
      if early_stop_count == patience:
        break

## 07-05 テストセット上で評価
* 見つけ出したベストな設定を使って、テストセット上での最終的な評価をおこなう。

In [ ]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_iterator, model, criterion)
print(f'\tLoss: {test_loss:.5f}(test)\t|\tAcc: {test_acc * 100:.2f}%(test)')